# Import and setup

In [1]:
import pandas as pd

In [2]:
# change display options
pd.options.display.max_rows = 150
pd.options.display.max_columns = 150

In [3]:
%%time
# import cleaned tourism and population data
tourism_pop_df = pd.read_csv('../data/derived/tourism_pop.csv', index_col=0)
tourism_pop_df.head(3)

CPU times: user 6.81 ms, sys: 2.75 ms, total: 9.56 ms
Wall time: 9.9 ms


,year_wb,country_id,country_txt_wb,country_code,arrivals,population
0,2012,8,Angola,AGO,528000,25107931
1,2012,5,Albania,ALB,3156000,2900401
2,2012,7,Andorra,AND,2238000,82427


In [4]:
tourism_pop_df.shape

(1078, 6)

The next line will throw an error about mixed types on some columns we don't plan to use:
* 62: `gsubname2`
* 64: `gsubname3`
* 80: `claimmode3_txt`
* 115: `divert`
* 116: `kidhijcountry`

In [5]:
%%time
# import cleaned terrorism data (may take a few minutes to load)
terrorism_df = pd.read_csv('../data/derived/terrorism.csv', index_col=0)
terrorism_df.head(3)

<string>:2: DtypeWarning: Columns (62,64,80,115,116) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 1.22 s, sys: 194 ms, total: 1.41 s
Wall time: 1.5 s


,eventid,year_gtd,month_gtd,day_gtd,approxdate,extended,resolution,country_id,country_txt_gtd,region,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative,alternative_txt,multiple,success,suicide,attacktype1,attacktype1_txt,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype1_txt,targsubtype1,targsubtype1_txt,corp1,target1,natlty1,natlty1_txt,targtype2,targtype2_txt,targsubtype2,targsubtype2_txt,corp2,target2,natlty2,natlty2_txt,targtype3,targtype3_txt,targsubtype3,targsubtype3_txt,corp3,target3,natlty3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,motive,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weaptype1,weaptype1_txt,weapsubtype1,weapsubtype1_txt,weaptype2,weaptype2_txt,weapsubtype2,weapsubtype2_txt,weaptype3,weaptype3_txt,weapsubtype3,weapsubtype3_txt,weaptype4,weaptype4_txt,weapsubtype4,weapsubtype4_txt,weapdetail,nkill,nkillus,nkillte,nwound,nwoundus,nwoundte,property,propextent,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related,nvictim,nvictimus,nvictimte
0,201201010001,2012,1,1,NaN,0,NaN,153,Pakistan,6,South Asia,Balochistan,Quetta,30.20082,66.994352,1.0,0,Incident occurred on Joint road.,01/01/2012: A grenade was thrown into a shop o...,1,1,1,0,NaN,NaN,0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,1,Business,2.0,Restaurant/Bar/Café,Unknown,Paan Shop,153.0,Pakistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Explosives,7.0,Grenade,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,12.0,0.0,0.0,1,3.0,Minor (likely < $1 million),-99.0,"The targeted shops, as well as several other s...",0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,There is doubt that this incident meets terror...,"""16 hurt in hand grenade attacks in Quetta, Hu...","""Grenade attack injures five in Quetta,"" Daily...","""Grenade attack injures nine in Quetta,"" Plus ...",START Primary Collection,-9,-9,0,-9,NaN,12.0,0.0,0.0
1,201201010002,2012,1,1,NaN,0,NaN,104,Kenya,11,Sub-Saharan Africa,North Eastern,Garissa,-0.45612,39.646123,1.0,0,NaN,01/01/2012: Assailants threw two grenades and ...,1,1,1,0,NaN,NaN,1,1,0,2,Armed Assault,NaN,NaN,NaN,NaN,1,Business,2.0,Restaurant/Bar/Café,Locus Pub,Bar,104.0,Kenya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Al-Shabaab,NaN,NaN,NaN,NaN,NaN,"The specific motive is unknown; however, Al-Sh...",1.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Explosives,7.0,Grenade,5.0,Firearms,2.0,Automatic or Semi-Automatic Rifle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Two grenades and AK-47 assault rifles were use...,3.0,0.0,0.0,14.0,0.0,0.0,1,3.0,Minor (likely < $1 million),-99.0,The dance floor was damaged in this attack.,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Casualty numbers for this attack represent an ...,"""BRIEF: Five killed in grenade attack in n...","""UPDATE 2-Gunmen attack bars in northeast Keny...","""Gunmen attack bars in northeast Kenya, at lea...",START Primary Collection,1,1,0,1,"201201010002, 201201010004",17.0,0.0,0.0
2,201201010004,2012,1,1,NaN,0,NaN,104,Kenya,11,Sub-Saharan Africa,North Eastern,Garissa,-0.45612,39.646123,1.0,0,NaN,01/01/2012: Assailants threw a grenade and ope...,1,1,1,0,NaN,NaN,1,1,0,2,Armed Assault,NaN,NaN,NaN,NaN,1,Business,2.0,Restaurant/Bar/Café,Unknown,Bar,104.0,Kenya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Al-Shabaab,NaN,NaN,NaN,NaN,NaN,"The specific motive is unknown; however, Al-Sh...",1.0,NaN,NaN,0,-99.0,0.

In [6]:
terrorism_df.shape

(59395, 138)

# Set indices

Before joining the datasets, I will set the indices t othe columns that they'll be joined on: `year` and `country_id`.

In [7]:
# rename year column
terrorism_df.rename(columns={'year_gtd':'year'}, inplace=True)

# set columns for join as indices
terrorism_df.set_index(['year', 'country_id'], inplace=True)

# sort by index
terrorism_df.sort_index(inplace=True)

# examine result
terrorism_df.head(3)

eventid  month_gtd  day_gtd approxdate  extended  \
year country_id                                                          
2012 4           201201020007          1        2        NaN         0   
     4           201201020010          1        2        NaN         0   
     4           201201020012          1        1        NaN         0   

                resolution country_txt_gtd  region  region_txt  provstate  \
year country_id                                                             
2012 4                 NaN     Afghanistan       6  South Asia      Logar   
     4                 NaN     Afghanistan       6  South Asia  Nangarhar   
     4                 NaN     Afghanistan       6  South Asia    Badghis   

                         city   latitude  longitude  specificity  vicinity  \
year country_id                                                              
2012 4              Puli Alam  34.002047  69.031149          1.0         1   
     4             Paindakhel  34.343559  67.921173          2.0         0   
     4           Bala Morghab  35.585669  63.328089          1.0         0   

                               location  \
year country_id                           
2012 4           Khwaja Azfal Qala area   
     4                              NaN   
     4                              NaN   

                                                           summary  crit1  \
year country_id                                                             
2012 4           01/02/2012: A civilian car hit a roadside bomb...      1   
     4           01/02/2012: A suicide bomber detonated near a ...      1   
     4           01/01/2012: Two assailants died when their min...      1   

                 crit2  crit3  doubtterr  alternative  alternative_txt  \
year country_id                                                          
2012 4               1      1          0          NaN              NaN   
     4               1      1          0          NaN              NaN   
     4               1      1          0          NaN              NaN   

                 multiple  success  suicide  attacktype1    attacktype1_txt  \
year country_id                                                               
2012 4                  0        1        0            3  Bombing/Explosion   
     4                  0        1        1            3  Bombing/Explosion   
     4                  0        1        0            3  Bombing/Explosion   

                 attacktype2 attacktype2_txt  attacktype3 attacktype3_txt  \
year country_id                                                             
2012 4                   NaN             NaN          NaN             NaN   
     4                   NaN             NaN          NaN             NaN   
     4                   NaN             NaN          NaN             NaN   

                 targtype1                targtype1_txt  targsubtype1  \
year country_id                                                         
2012 4                  14  Private Citizens & Property          67.0   
     4                   3                       Police          22.0   
     4                  14  Private Citizens & Property          67.0   

                                                targsubtype1_txt  \
year country_id                                                    
2012 4                              Unnamed Civilian/Unspecified   
     4           Police Building (headquarters, station, school)   
     4                              Unnamed Civilian/Unspecified   

                          corp1            target1  natlty1  natlty1_txt  \
year country_id                                                            
2012 4           Not Applicable          Civilians      4.0  Afghanistan   
     4            Afghan Police  Training Facility      4.0  Afghanistan   
     4           Not Applicable          Civilians      4.0  Afghanistan   

                 targtype2     

In [8]:
# rename year column
tourism_pop_df.rename(columns={'year_wb':'year'}, inplace=True)

# set columns for join as indices
tourism_pop_df.set_index(['year', 'country_id'], inplace=True)

# sort by index
tourism_pop_df.sort_index(inplace=True)

# examine result
tourism_pop_df.head(3)

country_txt_wb country_code  arrivals  population
year country_id                                                  
2012 5                 Albania          ALB   3156000     2900401
     6                 Algeria          DZA   2634000    37383887
     7                 Andorra          AND   2238000       82427

# Join datasets

I'll be doing a left join between these two datasets, to only keep countries and years that have tourism/population data. If a country and year is listed in `tourism_pop_df` and not `terrorism_df`, the columns related to terrorism will have NaN values.

In [9]:
# join tourism, population and terrorism data
df = tourism_pop_df.join(terrorism_df, how='left')
# reset index
df.reset_index(inplace=True)

In [10]:
df.shape

(18540, 142)

In [11]:
df.head(3)

,year,country_id,country_txt_wb,country_code,arrivals,population,eventid,month_gtd,day_gtd,approxdate,extended,resolution,country_txt_gtd,region,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative,alternative_txt,multiple,success,suicide,attacktype1,attacktype1_txt,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype1_txt,targsubtype1,targsubtype1_txt,corp1,target1,natlty1,natlty1_txt,targtype2,targtype2_txt,targsubtype2,targsubtype2_txt,corp2,target2,natlty2,natlty2_txt,targtype3,targtype3_txt,targsubtype3,targsubtype3_txt,corp3,target3,natlty3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,motive,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weaptype1,weaptype1_txt,weapsubtype1,weapsubtype1_txt,weaptype2,weaptype2_txt,weapsubtype2,weapsubtype2_txt,weaptype3,weaptype3_txt,weapsubtype3,weapsubtype3_txt,weaptype4,weaptype4_txt,weapsubtype4,weapsubtype4_txt,weapdetail,nkill,nkillus,nkillte,nwound,nwoundus,nwoundte,property,propextent,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related,nvictim,nvictimus,nvictimte
0,2012,5,Albania,ALB,3156000,2900401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012,6,Algeria,DZA,2634000,37383887,2.012012e+11,1.0,16.0,NaN,1.0,2012-01-17,Algeria,10.0,Middle East & North Africa,Biskra,Debdeb,34.840833,6.041389,1.0,1.0,NaN,01/16/2012: Three gunmen abducted Mohamed el A...,1.0,1.0,1.0,0.0,NaN,NaN,0.0,1.0,0.0,6.0,Hostage Taking (Kidnapping),NaN,NaN,NaN,NaN,2.0,Government (General),15.0,Politician or Political Party Movement/Meeting...,Government of Illizi,Prefect: Mohamed el Aid Khelfi,6.0,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Al-Qaida in the Islamic Maghreb (AQIM),NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,3.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,Firearms,5.0,Unknown Gun Type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,5.0,Successful Rescue,1.0,NaN,"""Algerian security official abducted near Liby...","""Algeria says kidnapped governor returns home,...","""Senior Algerian official kidnapped, taken tow...",START Primary Collection,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0
2,2012,6,Algeria,DZA,2634000,37383887,2.012022e+11,2.0,19.0,NaN,0.0,NaN,Algeria,10.0,Middle East & North Africa,Boumerdes,Unknown,36.759396,3.470261,4.0,0.0,Road 12 between Isser and Bordj Menaiel,"02/19/2012: A roadside bomb detonated, targeti...",1.0,1.0,1.0,0.0,NaN,NaN,0.0,1.0,0.0,3.0,Bombing/Explosion,NaN,NaN,NaN,NaN,4.0,Military,29.0,Military Unit/Patrol/Convoy,People's National Army (ANP),Convoy,6.0,Algeria,14.0,Private Citizens & Property,67.0,Unnamed Civilian/Unspecified,Not Applicable,Civilians,6.0,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,Explosives,12.0,Remote Trigger,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A remote-controlled roadside bomb consisting o...,4.0,0.0,0.0,19.0,0.0,0.0,1.0,3.0,Minor (likely < $1 million),-99.0,A passenger

In [12]:
# write to csv
df.to_csv('../data/derived/combined_data.csv')

# Group by incident

If an event occurs at separate times or locations, than they are considered separate incidents; see p. 12 of codebook. "If any discontinuity in time or space is noted, the event is comprised of multiple incidents."

If the event is part of a multiple incident, the variable `related` includes the text of event ID's from `eventid` for related incidents; see p. 13 and 17 of codebook.

I will group events into one incident if records are related, occurred in the same city, and occured on the same day. Numbers of fatalities, injured, and victims are summed across records.

In [13]:
df.related.count()

3340

In [14]:
# fill `related` column with `eventid` for singular events (needed for group by)
index_to_fill = df.index[df.eventid.notna() & df.related.isna()]
df.iloc[index_to_fill, 138] = df.iloc[index_to_fill, 6]

# check that `related` column is filled
df.related.count()

17942

In [15]:
df.shape

(18540, 142)

In [16]:
# group related incidents that occur on same day in same city
df_by_incident = df.groupby(['related',
                             'year','month_gtd','day_gtd',
                             'country_id','provstate','city'], dropna=False).agg(
    
    country_txt_wb  = ('country_txt_wb',  'first'),
    country_txt_gtd = ('country_txt_gtd', 'first'),
    country_code    = ('country_code',    'first'),
    region          = ('region',          'first'),
    region_txt      = ('region_txt',      'first'),
    arrivals        = ('arrivals',        'first'),
    population      = ('population',      'first'),
    incident_id     = ('eventid',         'first'),
    
    nkill     = ('nkill',     'sum'),
    nkillus   = ('nkillus',   'sum'),
    nkillte   = ('nkillte',   'sum'),
    nwound    = ('nwound',    'sum'),
    nwoundus  = ('nwoundus',  'sum'),
    nwoundte  = ('nwoundte',  'sum'),
    nvictim   = ('nvictim',   'sum'),
    nvictimus = ('nvictimus', 'sum'),
    nvictimte = ('nvictimte', 'sum'),
    event_ct_total   = ('eventid', 'count')
)

In [18]:
# reset index
df_by_incident.reset_index(inplace=True)

In [20]:
# drop columns
df_by_incident.drop(columns=['related','month_gtd','day_gtd','provstate','city'], inplace=True)

In [22]:
# inspect dataframe
df_by_incident.head()

,year,country_id,country_txt_wb,country_txt_gtd,country_code,region,region_txt,arrivals,population,incident_id,nkill,nkillus,nkillte,nwound,nwoundus,nwoundte,nvictim,nvictimus,nvictimte,event_ct_total
0,2012,92,India,India,IND,6.0,South Asia,6578000,1265782790,2.012010e+11,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1
1,2012,217,United States,United States,USA,1.0,North America,66966900,313830990,2.012010e+11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,2012,92,India,India,IND,6.0,South Asia,6578000,1265782790,2.012010e+11,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,1
3,2012,141,Nepal,Nepal,NPL,6.0,South Asia,803000,26989862,2.012010e+11,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1
4,2012,214,Ukraine,Ukraine,UKR,9.0,Eastern Europe,23013000,45593342,2.012010e+11,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,1


In [23]:
df_by_incident.shape

(17252, 20)

In [24]:
# write to csv
df_by_incident.to_csv('../data/derived/combined_data_by_incident.csv')

### Check out countries without incidents

In [25]:
df_by_incident[df_by_incident.event_ct_total==0]

,year,country_id,country_txt_wb,country_txt_gtd,country_code,region,region_txt,arrivals,population,incident_id,nkill,nkillus,nkillte,nwound,nwoundus,nwoundte,nvictim,nvictimus,nvictimte,event_ct_total
16654,2012,5,Albania,None,ALB,NaN,None,3156000,2900401,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
16655,2012,7,Andorra,None,AND,NaN,None,2238000,82427,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
16656,2012,8,Angola,None,AGO,NaN,None,528000,25107931,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
16657,2012,10,Antigua and Barbuda,None,ATG,NaN,None,247000,90409,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
16658,2012,12,Armenia,None,ARM,NaN,None,963000,2884229,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17247,2018,220,Vanuatu,None,VUT,NaN,None,116000,292680,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
17248,2018,225,Virgin Islands (U.S.),None,VIR,NaN,None,381000,106977,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
17249,2018,230,Zambia,None,ZMB,NaN,None,1072000,17351822,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
17250,2018,347,Timor-Leste,None,TLS,NaN,None,75000,1267972,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


# Group by year

In [27]:
# group by year and country
df_by_year = df_by_incident.groupby(['year','country_id']).agg(
    
    country_txt_wb  = ('country_txt_wb',  'first'),
    country_txt_gtd = ('country_txt_gtd', 'first'),
    country_code    = ('country_code',    'first'),
    region          = ('region',          'first'),
    region_txt      = ('region_txt',      'first'),
    arrivals        = ('arrivals',        'first'),
    population      = ('population',      'first'),
    
    nkill_total       = ('nkill', 'sum'),
    nkillus_total     = ('nkillus', 'sum'),
    nkillte_total     = ('nkillte', 'sum'),
    nwound_total      = ('nwound', 'sum'),
    nwoundus_total    = ('nwoundus', 'sum'),
    nwoundte_total    = ('nwoundte', 'sum'),
    nvictim_total     = ('nvictim', 'sum'),
    nvictimus_total   = ('nvictimus', 'sum'),
    nvictimte_total   = ('nvictimte', 'sum'),
    incident_ct_total = ('incident_id', 'count')
)

# reset index
df_by_year.reset_index(inplace=True)

# inspect dataframe
df_by_year.head()

,year,country_id,country_txt_wb,country_txt_gtd,country_code,region,region_txt,arrivals,population,nkill_total,nkillus_total,nkillte_total,nwound_total,nwoundus_total,nwoundte_total,nvictim_total,nvictimus_total,nvictimte_total,incident_ct_total
0,2012,5,Albania,None,ALB,NaN,None,3156000,2900401,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2012,6,Algeria,Algeria,DZA,10.0,Middle East & North Africa,2634000,37383887,25.0,0.0,2.0,76.0,0.0,0.0,101.0,0.0,2.0,27
2,2012,7,Andorra,None,AND,NaN,None,2238000,82427,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,2012,8,Angola,None,AGO,NaN,None,528000,25107931,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,2012,10,Antigua and Barbuda,None,ATG,NaN,None,247000,90409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [28]:
# write to csv
df_by_year.to_csv('../data/derived/combined_data_by_year.csv')